Autor: Juan Sebastián Vargas

Resolver la compuerta XOR con una red Neuronal

![](https://www.niser.ac.in/~smishra/teach/cs460/2020/lectures/lec19/Perceptron.jpg)

In [ ]:
import numpy as np
from sklearn.datasets import make_circles
import matplotlib.pyplot as plt

In [ ]:
class ActivationFunction():
  def __init__(self,act_f):
    if(act_f=="sig"):
      self.act_f= (lambda x: 1/(1+np.e**(-x))), (lambda x: x*(1-x))
    elif (act_f=="RELU"):
      self.act_f= (lambda x: np.maximum(0,x)), (lambda x: (x > 0) * 1) 

In [ ]:
class CostFunction():
  def __init__(self,cost_f):
    if(cost_f=="BCE"):
      self.cost_f= (lambda a,y: y*np.log(a)+(1-y)*np.log(1-a)), (lambda a,y: (-y/a)+((1-y)/(1-a))  )
    elif(cost_f=="SE"):
      self.cost_f= (lambda a,y: np.mean((y-a)**2)), (lambda a,y:(a-y))

In [ ]:
class NeuralLayer():
  def __init__(self,n_previous_neurons,n_neurons,act_f):
    self.n_previous_neurons=n_previous_neurons
    self.n_neurons=n_neurons
    self.act_f=act_f
    self.b= np.random.rand(n_neurons)
    self.w= np.random.rand(n_previous_neurons,n_neurons)
    self.z= np.zeros(n_neurons)
    self.a= np.zeros(n_neurons)
    self.act_f= ActivationFunction(act_f).act_f
    print(self.b)
    print(self.w)
    print(self.a)

  def updateActivationResult(self,input, isLast, epoch_index):
      #print("INPUT: ",input)
      #print("WT: ",self.w.T)
      #print("B: ",self.b)
      self.z= input @ self.w  + self.b
      print("Z VALUES: ", self.z)
      self.a= self.act_f[0](self.z)
      print("A VALUES: ", self.a)
      if(isLast):
        print("Predicción: ", self.a)


In [ ]:
class NeuralNetwork():
  def __init__(self, topography, activ_f, cost_f):
    self.topography=topography
    self.activ_f=activ_f
    self.cost_f=cost_f
    self._layers=[]
    for i in range(1,len(self.topography)):
      self._layers.append(NeuralLayer(self.topography[i-1],self.topography[i],self.activ_f))
    print("LAYER SIZE: ",len(self._layers))  

  def train(self,X,Y,lr,epoch):
      y_inflated= Y[:,np.newaxis]
      for e in range(0,epoch):
          #print(x_i[0],x_i[1], y_i)
          #print(".................FOWARD................")
          self.__feedFoward(X,y_inflated,e)
          #print(".................BACKPROPAGATION................")
          self.__backPropagation(X,y_inflated,lr)
          #print("\n\n----NEW X VALUES-------\n\n")

  def __feedFoward(self,x,y,epoch_index):
    self._layers[0].updateActivationResult(x,False,0)
    for i in range(1,len(self._layers)):
      #print("LAYER: ------- ",i)
      if(i==len(self._layers)-1):
        self._layers[i].updateActivationResult(self._layers[i-1].a,True,epoch_index)
      else:
        self._layers[i].updateActivationResult(self._layers[i-1].a,False,epoch_index)


  def __printNeuralNetwork(self):    
    print("*****************************NN****************")  
    for i in range(0,len(self._layers)):
      print("LAYER: ------- ",i)
      print("Z: ", self._layers[i].z)
      print("A: ", self._layers[i].a)
      print("-----------------")
    print("*****************************NN****************")  



  def __backPropagation(self,x,y,lr):
    #print("Y: ",y)
    #print("Activation in last layer: ",self._layers[-1].a, len(self._layers[-1].a))
    #print("Costo: ",CostFunction("SE").cost_f[0](self._layers[-1].a,y))
    #print("dC_da: ",dC_da)
    delta=0
    for i in reversed(range(0,len(self._layers))):
      #print("LAYER #: ",i)
      if(i==len(self._layers)-1):
        #print("LAST LAYER: ")
        #print("COST: ",CostFunction("SE").cost_f[0](self._layers[-1].a,y))
        #print("D_COST: ",CostFunction("BCE").cost_f[1](self._layers[-1].a,y))
        #print("ACTIVATION LAST LAYER: ",self._layers[-1].a)
        #print("W LAST LAYER: ",self._layers[-1].w)
        #print("B LAST LAYER: ",self._layers[-1].b)
        #print("Z LAST LAYER: ",self._layers[-1].z)
        #print("Z LAST LAYER: ",self._layers[-1].z)
        dC_da= CostFunction(self.cost_f).cost_f[1](self._layers[-1].a,y)


        #print("dC_da",dC_da.shape)

        da_dZ= self._layers[i].act_f[1](self._layers[-1].a) ### <<----- OJO MIRAR 1
        
      
        #print("da_dz",da_dZ.shape)

        delta=dC_da*da_dZ

        #print("I: ",i)
        #print("I-1: ",i-1)
        dZ_dW= self._layers[i-1].a 

        #print("Delta: ", delta.shape)

        gW= dZ_dW.T @ delta  ##Gradiente en W

        gb= np.mean(delta,axis=0, keepdims=True) ##Gradiente en b

        #print("W: ",self._layers[-1].w.shape)
        #print("FIRST LAYER gW" ,gW.shape)
        #print("gb" ,gb.shape)


        self._layers[-1].b= self._layers[-1].b - lr*(gb)
        self._layers[-1].w= self._layers[-1].w - lr*(gW)

      

      else:
        #print("OTHER LAYER, #: ",i)
      

        da_dZ= self._layers[i].act_f[1](self._layers[i].a) ### <<----- OJO MIRAR 2

        #print("Delta: ", delta.shape)
        #print(self._layers[i+1].w.shape)
        #print(da_dZ.shape)

        delta= (delta @ self._layers[i+1].w.T ) * da_dZ

        #print("New Delta: ", delta.shape)

        #print("I: ",i)
        #print("I-1: ",i-1)

        if(i==0):
          dZ_dW= x
        else:
          dZ_dW= self._layers[i-1].a   

        #print("dZ_dW: ", dZ_dW.shape)

        gW= dZ_dW.T @ delta  ##Gradiente en W

        #print(gW.shape)


        gb= np.mean(delta,axis=0, keepdims=True) ##Gradiente en b

        self._layers[i].b= self._layers[i].b - lr*(gb)
        self._layers[i].w= self._layers[i].w - lr*(gW)

        
        
   



In [ ]:
x_train=np.array([
    [1,1],
    [1,0],
    [0,0],
    [0,1],
])
y_train= np.array([0,1,0,1])

In [ ]:
model=NeuralNetwork([2,2,1],"RELU","BCE")

[0.57107471 0.78168651]
[[0.11666017 0.64091598]
 [0.4575166  0.05526268]]
[0. 0.]
[0.20343467]
[[0.20348811]
 [0.85292141]]
[0.]
LAYER SIZE:  2


In [ ]:
model.train(x_train,y_train,0.00001,10)

Z VALUES:  [[1.14525148 1.47786517]
 [0.68773488 1.42260249]
 [0.57107471 0.78168651]
 [1.02859131 0.83694918]]
A VALUES:  [[1.14525148 1.47786517]
 [0.68773488 1.42260249]
 [0.57107471 0.78168651]
 [1.02859131 0.83694918]]
Z VALUES:  [[1.69698258]
 [1.55674867]
 [0.98635875]
 [1.12659266]]
A VALUES:  [[1.69698258]
 [1.55674867]
 [0.98635875]
 [1.12659266]]
Predicción:  [[1.69698258]
 [1.55674867]
 [0.98635875]
 [1.12659266]]
